# Face Recognition
https://towardsdatascience.com/real-time-face-recognition-an-end-to-end-project-b738bb0f7348
https://github.com/Mjrovai/OpenCV-Face-Recognition/blob/master/FaceDetection/faceDetection.py
https://face-recognition.readthedocs.io/en/latest/face_recognition.html

## Objective
The objective is to use a Raspberry Pi 3+ and the camera for the Raspbery Pi and have it act as a security camera with face dection features. The Raspberry Pi should work headlessly, be able to collect faces for training and dectect people's faces that has been trained. We will be using the lastest version of Raspian as of 2/15/2021 as well as OpenCV and xxx. The majority of this walk through was gathered from different articles and videos that will be linked below. Although I didnt go through the OpenCV documentations, this was a good start and at some point I would like to do a complete OpenCV walk through.

Limitations: Exploring OpenCV and the pi's camera was fun but there were some limitations. The pi's camera to wasn't able to capture people's faces from the distance that I wanted it to. The idea was to have it sit on a window sile and gather faces as people walked by. The pi had difficulty capturing people's faces from about 50 feet away and for the face capturing portion, your subject had to be still for several seconds looking directly at the camera. Best results came at about 2-3 feet, anything between 3-6 feet would be identified as "Unknonw" and at 6 feet, it wouldnt pick up any faces at all. I would like to try this again but with a high quality usb camera or with a better pi camera.

This tutorial will be broken down in many parts listed below:

1. [Video with Python](#1.Video-with-Python)
2. [Facial Recognition with Video](#2.Facial-Recognition-with-Video)
3. [Data Gathering](#3.Data-Gathering)
4. [Training](#4.Training)
5. [Recognizer](#5.Recognizer)


Follow up:
 - From command line, show the face distance and adjust tolerance, @4:30 of youtube video
 - Go through OpenCV doc
 - Try same setup with usb camera to better capture training data
 

## 1.Video with Python
[Top](#Objective)
<br>
<br>
`pip3 install opencv-python`
<br>
and execute the following code. Here we are just initiating the camera, using opencv/
<br>


In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0) 
cap.set(3,640) # 3 set Width
cap.set(4,480) # 4 set Height

while(True):
    ret, frame = cap.read()
    #frame = cv2.flip(frame, -1) # Flip camera vertically
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #sets the frame to gray sale
    
    cv2.imshow('frame', frame) #this shows the "frame" frame from the above ret, frame
    #cv2.imshow('gray', gray)  #this shows the converted gray scale frame. When we enable both we will show two frames, one in color one is grey scale
    
    k = cv2.waitKey(30) & 0xff #dont know how this code works exactly but it kills the process when you press ESC
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) /tmp/pip-req-build-ddpkm6fn/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


The code output above is pretty simple. We load the face_recognition package, find the location of the all faces in a picture that we pass using the .face_location function. The function returns a tuple of the location of the square boxes around each person's face,  (top, right, bottom, left). There are 5 faces so five items in our tuple.

## 2.Facial Recognition with Video
[Top](#Objective)
<br>
<br>
To detect faces we need to use the Haar Cascade classifier. Using Haar feature-base we can detect faces as well as objects. You can create your own cascade classifier or use defualt ones. You can find an assortment of Haar classifiers here:

https://github.com/opencv/opencv/tree/master/data/haarcascades
<br>
<br>
1)Set opencv to capture video
<br>
2)Create loop that will read the frames
<br>
3)Change the frame to grayscale
<br>
4)Use a cascade to find the face
<br>
5)Draw rectangles around the frame

In [11]:
import numpy as np
import cv2

faceCascade = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')

#faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,640) # 3 set Width
cap.set(4,480) # 4 set Height

while True:
    ret, img = cap.read() #Returns a bool T/F is frame is read correctly
    #img = cv2.flip(img, -1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert image to gray color space
    faces = faceCascade.detectMultiScale(        #faceCascade OBJECT has a METHOD call detectMultiScale, this runs a classifier cascade over the image
        gray,                                    #MultiScale algorithm looks at a subregion of an image in multiple scales to detect faces of varying size
        scaleFactor=1.2,                         #How much to resize the image, lower the better, because you reduce the size of the image by a smaller amounmt, 
        minNeighbors=5,                                  #to match the size of the model, reduce it too much and you loose lots of data from the pic?     
        minSize=(20, 20)                                  #minNeighbor says how many neighbor each candidate rectangle should retain. Higher value, fewer resutls but higher qaulity 3-6 is good
    )                                                     #Min size of possible objects. Small than this we ignore. Return x,y, start loction, h hieight and w width
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)  #.rect(image, start.point, end.point, color, thickness)), how to make the rectangle
        roi_gray = gray[y:y+h, x:x+w]  #dont know why indexing the gray image
        roi_color = img[y:y+h, x:x+w]  #dont know why indexing of the color image
    cv2.imshow('video',img)            #call cv2 to show, a windwing called video, and the img we cap.read() from above
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()

The faces do not match


### 3.Data Gathering
[Top](#Objective)
<br>
<br>
Here we will be gathering the faces for so we can identify the faces. We are going to setup the camera to captures the individual faces and save them for training. !!!When using this script to gather faces, best to use number as the trainer is setup to use id numbers not names
<br>
<br>
1)Set opencv to capture video
<br>
2)Create loop that will read the frames
<br>
3)Change the frame to grayscale
<br>
4)Use a cascade to find the face, then save all the faces to a directory

In [12]:
import numpy as np
import cv2
import os

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

face_detector = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')

#for each person, enter one numeric face id

face_id = input('\n enter user id end press <return> ==> ')
print("\n [INFO] Initializing face capture. Look at the camera and wait..")

count = 0
while True:
    ret, img = cap.read()
    #img = cv2.flip(img, -1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        count +=1
        #Save the captured image into the dataset foldera
        cv2.imwrite("../face.sample/User."+str(face_id)+'.'+str(count)+".jpg",gray[y:y+h,x:x+w])
        cv2.imshow('image',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break
    elif count >= 30: #Take 30 face sample and stop video
        break

cap.release()
cv2.destroyAllWindows()

### 4.Training
[Top](#Objective)
<br>
<br>
Here we will be using .createLBPHFacesRecognizer and the .CascadeClassifier as we did in the previous example, more information about the two can be found here (https://stackoverflow.com/questions/8791178/haar-cascades-vs-lbp-cascades-in-face-detection). We are going to take faces we gathered above and match it to an image with unknown faces and hopefully it can identify the face in the unknonw image. It will return a trainer.yml that we will use later to recognize new faces. This works very much like the last we codes we ran:
<br>
1)Set opencv to capture video
<br>
2)Create loop that will read the frames
<br>
3)Change the frame to grayscale
<br>
4)Use a cascade to find the face
<br>
5)Draw rectangles around the frame

In [39]:
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = '../face.sample'
recognizer = cv2.face.createLBPHFaceRecognizer() #Local Binary Patterns Histogram face recognizer in opencCV
detector = cv2.CascadeClassifier("../haarcascade_frontalface_default.xml") #detect face or any objects

# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)] #Creates a list of each image and its path
    faceSamples=[]                  #os.path.join() will take the path variable which is our parent path (../face.sample) to our currnet directory 
    ids = []                        #Then combines it with with each of the files in the parent path using os.listdir, ../face.sample
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # Convert image in from path to a PIL image format, here convert to gray scale "L"
        img_numpy = np.array(PIL_img,'uint8')   #Convert the PIL image to a numpy unit8 array
        id = int(os.path.split(imagePath)[-1].split(".")[1])  #os.path.split(), splits the path into the directory path, and the file. 
        faces = detector.detectMultiScale(img_numpy)   #from os.path.split() we take the filename, [-1], then split the file name and take the [1] position, which is the id we gave it before
        for (x,y,w,h) in faces:    #detector.detectMultiScale() is simialr to face_recognition.face_location in that it return x,y,w,h top, right, left bottom
            faceSamples.append(img_numpy[y:y+h,x:x+w])  #the location from faces is now applied to the numpy image, uint8
            ids.append(id)  #then append the id we got from the split os path for id
    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")

faces,ids = getImagesAndLabels(path)  #from our function we assign what is returned to faces and ids
recognizer.train(faces, np.array(ids)) #then train the faces to ids?

# Save the model into trainer/trainer.yml
recognizer.save('../trainer.yml') 

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

[0.99205385 0.89515966]
[0.48802559 0.75429731]
[0.96776413 0.93438534]
[0.79075103 0.50995554]
[0.92704192 0.99004671]


### 5.Recognizer
[Top](#Objective)
<br>
<br>
Finally we will capture a face and have our recognizer make a prediction and returns its id and index.
<br>
<br>
0) Initialize video capture using cv2
<br>
1) Load image file of known (bruno)
<br>
2) Encode the face to get a list of 128-dimensional face encodings (bruno)
<br>
3) Assign the face encoding names (bruno)
<br>
4) Start a "loop" using while to capture all the frames from the camera
<br>
5) Get face location for the unknown face using face_recognition.face_locations()
<br>
6) Encode the unknown faces using the location from #5 with .face_encoding()
<br>
7) Create a for loop to iterate throught each video frame
<br>
8) Check to see if the face match
<br>
9) Set a rectangle around the face with name
<br>
https://www.youtube.com/watch?v=lC_y8wD7F3Y&t=797s

In [32]:
#Issues you ran into
#1 when you curled the cascade, it did not download completely, you had to go to the website and dl and extract
#the cv2.face.LBPHFaceRecognizer() we either an older or new package, had to use cv2.face.createLBPHFaceRecognizer()
#with that we had to use save instead of write and load instead of read

import cv2
import numpy as np
import os 

recognizer = cv2.face.createLBPHFaceRecognizer() #Local Binary Patterns Histogram face recognizer in opencCV. Assigns recognizer to a face recognizer
recognizer.load('../trainer.yml') #loads the trainer.yml file we got from above to recognizer we just assign
cascadePath = "../haarcascade_frontalface_default.xml" #set cascade xml path
faceCascade = cv2.CascadeClassifier(cascadePath); #like we did above for dectector
font = cv2.FONT_HERSHEY_SIMPLEX #sets the font

#iniciate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['Linc','hman'] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

while True:
    ret, img =cam.read()
    #img = cv2.flip(img, -1) # Flip vertically
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #convert image to gray color space
    
    faces = faceCascade.detectMultiScale(  #https://stackoverflow.com/questions/36218385/parameters-of-detectmultiscale-in-opencv-using-python
        gray,                                    #MultiScale algorithm looks at a subregion of an image in multiple scales to detect faces of varying size
        scaleFactor=1.2,                         #How much to resize the image, lower the better, because you reduce the size of the image by a smaller amounmt, 
        minNeighbors=5,                          #to match the size of the model, reduce it too much and you loose lots of data from the pic?     
        minSize = (int(minW), int(minH))         #minNeighbor says how many neighbor each candidate rectangle should retain. Higher value, fewer resutls but higher qaulity 3-6 is good                                                     
                )                                #Min size of possible objects. Small than this we ignore. Return x,y, start loction, h hieight and w width
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2) #.rect(image, start.point, end.point, color, thickness)), how to make the rectangle
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w]) #the recognizer will predict the image and its ide and return the id and confidenc
     
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(              #Put text around the square we made to find the faces
                    img,          #Image we've put the box around
                    str(id),      #The id we got from the if statements, unknow or unknown
                    (x+5,y-5),   #The position to put the ext
                    font,        #Font type as describe above
                    1,           #Fontscale
                    (255,255,255), #Color
                    2              #Thickness
                   )
        cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
    
    cv2.imshow('camera',img) #name of the window and the img to show
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [ ]:
### #.asdf
[Top](#Objective)
<br>
<br>